## Using the StableBaselines3 library for reinforcement learning

In this notebook we test an implementation of the proximal policy optimization (PPO)
PPO is described in detail in https://arxiv.org/abs/1707.06347. It is a variant of Trust Region Policy Optimization (TRPO) described (in this paper )[https://arxiv.org/abs/1502.05477]. The PPO algorithm works in two phases. In one phase, a large number of rollouts are performed (in parallel). The rollouts are then aggregated on the driver and a surrogate optimization objective is defined based on those rollouts. We then use SGD to find the policy that maximizes that objective with a penalty term for diverging too much from the current policy.

![ppo](https://raw.githubusercontent.com/ucbrise/risecamp/risecamp2018/ray/tutorial/rllib_exercises/ppo.png)

We begin by installing Python 3.8 in our environment, mounting Google Drive and cloning the repository with the 3D bin packing environment

In [1]:
!sudo apt-get install python3.8

#change alternatives
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 2

#check python version
!python --version

#install pip
!sudo apt-get install python-pip
!sudo apt install python3.8-distutils

#force reinstall pip (fixes issue with pip not working)
#see: https://askubuntu.com/questions/1025189/pip-is-not-working-importerror-no-module-named-pip-internal
!curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!python3 get-pip.py --force-reinstall



Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libpython3.8-minimal libpython3.8-stdlib python3.8-minimal
Suggested packages:
  python3.8-venv binfmt-support
The following NEW packages will be installed:
  libpython3.8-minimal libpython3.8-stdlib python3.8 python3.8-minimal
0 upgraded, 4 newly installed, 0 to remove and 20 not upgraded.
Need to get 4,695 kB of archives.
After this operation, 18.5 MB of additional disk space will be used.
Get:1 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic/main amd64 libpython3.8-minimal amd64 3.8.14-1+bionic1 [762 kB]
Get:2 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic/main amd64 python3.8-minimal amd64 3.8.14-1+bionic1 [1,839 kB]
Get:3 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic/main am

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
%cd /content/drive/MyDrive
! rm -rf /content/drive/MyDrive/3D-bin-packing
!git clone https://github.com/luisgarciar/3D-bin-packing.git
%cd 3D-bin-packing/
!git checkout dev-luis

/content/drive/MyDrive
Cloning into '3D-bin-packing'...
remote: Enumerating objects: 400, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 400 (delta 3), reused 8 (delta 1), pack-reused 369
Receiving objects: 100% (400/400), 8.84 MiB | 12.13 MiB/s, done.
Resolving deltas: 100% (211/211), done.
/content/drive/MyDrive/3D-bin-packing
Branch 'dev-luis' set up to track remote branch 'dev-luis' from 'origin'.
Switched to a new branch 'dev-luis'


We add the path of the repository to be able to use the files and modules.

In [3]:
import sys
sys.path.append('/content/drive/MyDrive/3D-bin-packing')

We now install the required libraries. We need a special version of the library 'stable-baselines-3' compatible with OpenAI Gym version >=0.24.

In [4]:
!pip install git+https://github.com/carlosluis/stable-baselines3@fix_tests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/carlosluis/stable-baselines3 (to revision fix_tests) to /tmp/pip-req-build-5lcodqdf
  Running command git clone -q https://github.com/carlosluis/stable-baselines3 /tmp/pip-req-build-5lcodqdf
  Running command git checkout -b fix_tests --track origin/fix_tests
  Switched to a new branch 'fix_tests'
  Branch 'fix_tests' set up to track remote branch 'fix_tests' from 'origin'.
     |████████████████████████████████| 720 kB 2.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for stable-baselines3: filename=stable_baselines3-1.6.1a4-py3-none-any.whl size=168726 sha256=7101999df9306cd743ed2b4cb5f18945bfb357fc33794d7130fb87dd9d45862e
  Stored in directory: /tmp/pip-ephem-wheel-cache-foabh4k6/wheels/f5/52/9d/2632ccdaa26fb5f7138d7d7c6610afc2faf6cd6e119de5c703
  Created w

We complete the instalation of the rest of the required libraries with the requirements.txt file from the repository.

In [6]:
!pip install -r /content/drive/MyDrive/3D-bin-packing/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached nptyping-2.3.1-py3-none-any.whl (32 kB)
  Using cached numpy-1.21.5-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Using cached Pillow-9.2.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Using cached plotly-5.9.0-py2.py3-none-any.whl (15.2 MB)
  Using cached pytest-7.1.2-py3-none-any.whl (297 kB)
  Using cached sb3_contrib-1.6.0-py3-none-any.whl (78 kB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: pluggy
    Found existing installation: pluggy 0.7.1
    Uninstalling pluggy-0.7.1:
      Successfully uninstalled pluggy-0.7.1
  Attempting uninstall: pytest
    Found existing installation: pytest 3.6.4
    Uninstalling pytest-3.6.4:
      Successfully uninstalled pytest-3.6.4
  Attempting uninstall: plotl

Add the path 

In [13]:
import sys
import os

py_file_location = "/content/drive/MyDrive/3D-bin-packing"
sys.path.append(os.path.abspath(py_file_location))

In [14]:
import src

/usr/local/lib/python3.7/dist-packages/tornado/httputil.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  class HTTPHeaders(collections.MutableMapping):


We now test the PPO algorithm with the 3D bin packing environment.

In [ ]:
import gym
from gym import make
import warnings
import sb3_contrib
from sb3_contrib.ppo_mask import MultiInputPolicy
from sb3_contrib.common.wrappers import ActionMasker
from sb3_contrib.ppo_mask import MaskablePPO
from stable_baselines3.common.callbacks import CallbackList, CheckpointCallback, EvalCallback
import numpy as np
from numpy.typing import NDArray
from src.utils import boxes_generator

import warnings

def mask_fn(env: gym.Env) -> NDArray:
    # Do whatever you'd like in this function to return the action mask
    # for the current env. In this example, we assume the env has a
    # helpful method we can rely on.
    return env.get_action_mask


warnings.filterwarnings("ignore", category=DeprecationWarning)

container_size = [10, 10, 10]
box_sizes1 = boxes_generator(container_size, num_items=64, seed=42)
box_sizes2 = boxes_generator(container_size, num_items=64, seed=5)

# Environment initialization
train_env = make(
    "PackingEnv-v0",
    container_size=container_size,
    box_sizes=box_sizes1,
    num_visible_boxes=3,
    render_mode=None,
    options=None,
)
obs = train_env.reset()

# Environment for evaluation every 500 iterations
eval_env = gym.make("PackingEnv-v0",
    container_size=container_size,
    box_sizes=box_sizes2,
    num_visible_boxes=3,
    render_mode=None,
    options=None,)

# MaskablePPO initialization
# To configure the Maskable PPO agent, we need to wrap the environments
train_env = ActionMasker(train_env, mask_fn)  # Wrap to enable masking
eval_env = ActionMasker(eval_env, mask_fn)

# Define callback - Save a checkpoint every 100 steps
checkpoint_callback = CheckpointCallback(save_freq=10, save_path='../logs/')
eval_callback = EvalCallback(eval_env, best_model_save_path='../logs/best_model',
                             log_path='./logs/results', eval_freq=15)

print("begin training")
model = MaskablePPO("MultiInputPolicy", train_env, gamma=0.4, verbose=1)
model.learn(5000)
print("done training")
model.save("../models/small_env_v0")


/opt/anaconda3/envs/3D-bin-packing/lib/python3.9/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, '__version__') or LooseVersion(tensorboard.__version__) < LooseVersion('1.15'):


begin training
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
import gym